In [183]:
import cv2
import random
import numpy
import math
import statistics
from PIL import Image, ImageOps
from tqdm import tqdm
import scipy.linalg
from collections import Counter
from scipy.spatial import distance
from numpy.linalg import norm
import warnings
warnings.simplefilter("ignore", RuntimeWarning)

In [184]:
img_read = Image.open(r"Data_SLIC_img.png") 
count_rows, count_cols = 256, 256
img_read = img_read.resize((count_rows, count_cols))
img_array = numpy.array(img_read)
temp_pad_arr = numpy.pad(img_array, pad_width=1, mode='constant', constant_values=0)
list_rgb = []
list_xy = []
for i in range(count_rows):
    for j in range(count_cols):
        list_xy.append([j/255,i/255])
        r_val,g_val,b_val = img_array[i][j][0], img_array[i][j][1], img_array[i][j][2]
        list_vals = [r_val, g_val, b_val]
        list_rgb.append(list_vals)

np_rgb_pix = numpy.array(list_rgb)
np_xy_pix = numpy.array(list_xy)

In [185]:
value_N = count_rows * count_cols
value_k = 125
value_S = math.sqrt(value_N / value_k)
value_m = 10
#Now choosing the initial superpixel centres
int_S = (int)(value_S)
list_clust_cent = []
for x in range(int_S, count_rows, +int_S):
    for y in range(int_S, count_cols, +int_S):
        list_clust_cent.append([x, y])
        
#Now finding the pixel with lowest gradient in the 3 X 3 neighbourhood
list_final_clust = []
for c in list_clust_cent:
    curr_row, curr_col = c[0], c[1]
    list_grad_values, list_pixels_neigh = [], []
    for i in range(curr_row-1, curr_row+2, +1):
        for j in range(curr_col-1, curr_col+2, +1):
            list_pixels_neigh.append([i, j])
            rx = img_array[i+1][j][0]-img_array[i-1][j][0]
            gx = img_array[i+1][j][1]-img_array[i-1][j][1]
            bx = img_array[i+1][j][2]-img_array[i-1][j][2]
            list_x_grad = [rx,gx,bx]
            np_list_x_grad = numpy.array(list_x_grad)
            val_1 = (norm(np_list_x_grad)) ** 2
            
            ry = img_array[i][j+1][0]-img_array[i][j-1][0]
            gy = img_array[i][j+1][1]-img_array[i][j-1][1]
            by = img_array[i][j+1][2]-img_array[i][j-1][2]
            list_x_grad = [rx,gx,bx]
            np_list_x_grad = numpy.array(list_x_grad)
            val_2 = (norm(np_list_x_grad)) ** 2
            val_grad = val_1 + val_2
            
            list_grad_values.append(val_grad)
    
    #Now finding the pixel in neighbourhood with minimum gradient value
    idx_min = list_grad_values.index(min(list_grad_values))
    list_final_clust.append(list_pixels_neigh[idx_min])
    

In [186]:
#Finding the x-y values od selected cluster centres
list_xy_clust = []
list_rgb_clust = []
for cl in list_final_clust:
    i,j = cl[0], cl[1]
    list_xy_clust.append([cl[1]/255, cl[0]/255])
    r_val,g_val,b_val = img_array[i][j][0], img_array[i][j][1], img_array[i][j][2]
    list_vals = [r_val, g_val, b_val]
    list_rgb_clust.append(list_vals)
    
np_xy_clust = numpy.array(list_xy_clust)
np_rgb_clust = numpy.array(list_rgb_clust)
#Storing the x-y values of 

In [187]:
#Assigning the pixles to clusters and finding all points in the clusters
pixel_num = -1
value_threshold = 2
dict_pix_clust_map = {}
dict_clust_pix_map = {}
for h in range(len(list_final_clust)):
    dict_clust_pix_map[h] = []

count_iterations = 0
while True:
    dict_clust_pix_map = {}
    for h in range(len(list_final_clust)):
        dict_clust_pix_map[h] = []
    count_iterations += 1
    pixel_num = -1
    for i in range(count_rows):
        for j in range(count_cols):
            pixel_num += 1
            list_distances = []
            for k in range(np_xy_clust.shape[0]):
                cl = np_xy_clust[k]
                diff_x = abs(np_xy_pix[pixel_num][0] - cl[0])
                diff_y = abs(np_xy_pix[pixel_num][1] - cl[1])
                diff_x *= 255
                diff_y *= 255
                if diff_x <= int_S and diff_y <= int_S:
                    dist_rgb =  distance.euclidean(np_rgb_clust[k], np_rgb_pix[pixel_num])
                    dist_xy = distance.euclidean(np_xy_clust[k], np_xy_pix[pixel_num])
                    total_dist = dist_rgb + ((value_m / int_S) * dist_xy)
                    list_distances.append([k, total_dist])

            if len(list_distances) >= 1:
                list_distances.sort(key=lambda x:x[1])
                idx_clust_nearest = list_distances[0][0]
                dict_clust_pix_map[idx_clust_nearest].append(pixel_num)
                dict_pix_clust_map[pixel_num] = idx_clust_nearest

    #Now recomputing the cluster centres
    flag_var = 0
    for clust in dict_clust_pix_map:
        list_r, list_g, list_b = [np_rgb_clust[clust][0]],[np_rgb_clust[clust][1]],[np_rgb_clust[clust][2]]
        list_x, list_y = [np_xy_clust[clust][0]],[np_xy_clust[clust][1]]
        for p in dict_clust_pix_map[clust]:
            list_r.append(np_rgb_pix[p][0])
            list_g.append(np_rgb_pix[p][1])
            list_b.append(np_rgb_pix[p][2])
            list_x.append(np_xy_pix[p][0])
            list_y.append(np_xy_pix[p][1])
        
        list_comp = [list_r,list_g,list_b,list_x,list_y]
        np_list_comp = numpy.array(list_comp)
        cord_new_clust = numpy.mean(np_list_comp, axis=1)
        diff_r = abs(cord_new_clust[0] - np_rgb_clust[clust][0])
        diff_g = abs(cord_new_clust[1] - np_rgb_clust[clust][1])
        diff_b = abs(cord_new_clust[2] - np_rgb_clust[clust][2])
        diff_x = abs(cord_new_clust[3] - np_xy_clust[clust][0])
        diff_y = abs(cord_new_clust[4] - np_xy_clust[clust][1])
        #Now check if the L1 distance is greater than threshold
        total_diff = diff_r + diff_g + diff_b + diff_x + diff_y
        if (diff_x + diff_y) > value_threshold:
            flag_var = 1
            
        np_temp_1 = numpy.copy(cord_new_clust[:3])
        np_temp_2 = numpy.copy(cord_new_clust[3:])
        np_rgb_clust[clust] = np_temp_1
        np_xy_clust[clust] = np_temp_2
    
    #Check thethreshold condition
    if flag_var == 0 or count_iterations == 30:
        break

In [188]:
#Now doing for disconnected regions
#First doing for unlabelled ones
for p in range(total_val):
    if dict_pix_clust_map.get(p) is None:
        list_neigh_clust = []
        p_x = np_xy_pix[p][0]
        p_y = np_xy_pix[p][1]
        for q in range(np_xy_pix.shape[0]):
            dist_x = abs(p_x - np_xy_pix[q][0])
            dist_y = abs(p_y - np_xy_pix[q][1])
            dist_x *= 255
            dist_y *= 255
            if dist_x <= int_S and dist_y <= int_S:
                if dict_pix_clust_map.get(q) is not None:
                    list_neigh_clust.append(dict_pix_clust_map[q])
        
        len_half = (len(list_neigh_clust)) / 2
        counter_clust = Counter(list_neigh_clust)
        list_pairs_clust = []
        for c in counter_clust:
            list_pairs_clust.append([c, counter_clust[c]])
        
        list_pairs_clust.sort(key=lambda x:x[1], reverse=True)
        max_clust_cnt = list_pairs_clust[0][1]
        dict_pix_clust_map[p] = list_pairs_clust[0][0]
            
copy_img_arr = numpy.copy(img_array)
for pix in dict_pix_clust_map:
    clust = dict_pix_clust_map[pix]
    pix_x = (int)(np_xy_pix[pix][1] * 255)
    copy_img_arr[pix_x][pix_y][0] = (np_rgb_clust[clust][0])
    copy_img_arr[pix_x][pix_y][1] = (np_rgb_clust[clust][1])
    copy_img_arr[pix_x][pix_y][2] = (np_rgb_clust[clust][2])

new_img_slic = Image.open(r"Q2_SLIC_superpixels.png") 
new_img_slic.show()

In [190]:
dict_tuple_idx = {}
num_pixel = -1
total_val = count_rows * count_cols
        
for i in range(count_rows):
    for j in range(count_cols):
        num_pixel += 1
        cord_tup = (i,j)
        dict_tuple_idx[cord_tup] = num_pixel

copy_img_arr_2 = numpy.copy(img_array)
end_r = count_rows-1
end_c = count_cols-1
for i in range(1, end_r):
    for j in range(1, end_c):
        list_neigh = [dict_tuple_idx[(i,j-1)],dict_tuple_idx[(i,j+1)],dict_tuple_idx[(i-1,j)],dict_tuple_idx[(i+1,j)]]
        list_clust_belong = []
        for h in list_neigh:
            list_clust_belong.append(dict_pix_clust_map[h])
        set_list_clust_bel = set(list_clust_belong)
        if len(set_list_clust_bel) > 1:
            copy_img_arr_2[i][j][0] = 0
            copy_img_arr_2[i][j][0] = 255
            copy_img_arr_2[i][j][0] = 0

In [191]:
new_img_slic_bound = Image.fromarray(copy_img_arr_2)
new_img_slic_bound.save("Result_SLIC_boundary.png")
new_img_slic_bound.show()